<a href="https://colab.research.google.com/github/hjfuentes/IAGenerativa/blob/main/Microsoft/AI_foundry_MAF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#%pip install -q "mcp>=1.23.1" agent-framework azure-ai-projects python-dotenv
%pip install -q --pre azure-ai-projects>=2.0.0b1 azure-identity

In [ ]:
# ============================================================
# 1) IMPORTS Y CONFIGURACIÓN INICIAL
# ============================================================

import os

# ------------------------------------------------------------
# Lectura de credenciales desde un archivo de texto
# El archivo debe tener EXACTAMENTE 3 líneas en este orden:
#   1) AZURE_CLIENT_ID
#   2) AZURE_CLIENT_SECRET
#   3) AZURE_TENANT_ID
# ------------------------------------------------------------

with open("/content/aifoundrymaf.txt", "r") as archivo:
  lineas = archivo.read().splitlines()

# Desempaquetado: asigna cada línea a una variable
azure_client_id, azure_client_secret, azure_tenant_id = lineas

# ------------------------------------------------------------
# Variables de entorno para autenticación con Azure
# DefaultAzureCredential() leerá estas variables
# ------------------------------------------------------------
os.environ["AZURE_CLIENT_ID"]=azure_client_id
os.environ["AZURE_CLIENT_SECRET"]=azure_client_secret #certificado y secretos: clave-foundry-colab
os.environ["AZURE_TENANT_ID"]=azure_tenant_id

# ------------------------------------------------------------
# Endpoint del proyecto en Azure AI Foundry
# IMPORTANTE: Debe ser el endpoint del proyecto correcto
# ------------------------------------------------------------
###os.environ["AZURE_AI_FOUNDRY_PROJECT_ENDPOINT"] ="https://mac-foundry-agent.services.ai.azure.com/api/projects/proj-agent"
os.environ["AZURE_AI_FOUNDRY_PROJECT_ENDPOINT"] ="https://modelo-foundry-colab.services.ai.azure.com/api/projects/proj-modelo-foundry-colab"

# ------------------------------------------------------------
# Parámetros de configuración del agente
# agent_name: nombre lógico del agente (reutilizable)
# model_id: modelo que usará el agente
# embedding_id: se usará cuando agregues RAG / embeddings
# ------------------------------------------------------------
agent_name = "CODX-agentSolution-qa01"
model_id = "gpt-4.1-mini"
embedding_id="text-embedding-3-small"

In [ ]:
# ============================================================
# 2) CREACIÓN DEL AGENTE (MAF + AI FOUNDY)
# ============================================================

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition
from dotenv import load_dotenv

# ------------------------------------------------------------
# Crear el cliente del proyecto de AI Foundry
# credential=DefaultAzureCredential() usará las variables de entorno
# ------------------------------------------------------------
project_client = AIProjectClient(
    endpoint=os.environ["AZURE_AI_FOUNDRY_PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

# ------------------------------------------------------------
# Crear (o versionar) un agente tipo Prompt Agent
# - create_version crea una nueva versión del agente
# - instructions actúan como "system prompt" persistente
# ------------------------------------------------------------
agent = project_client.agents.create_version(
    agent_name=agent_name,
    definition=PromptAgentDefinition(
        model=model_id,
        instructions=""" Eres un asistente amable y especializado en matemáticas
         no debes responder preguntas de otro tema e indicar que no estas habilitado para dar información de un tema distinto, se breve"""
    ),
)

# Mostrar datos relevantes del agente creado/versionado
print(f"Agene creado (id: {agent.id}, nombre: {agent.name}, version: {agent.version})")


Agent created (id: CODX-agentSolution-qa01:2, name: CODX-agentSolution-qa01, version: 2)


In [ ]:
# ============================================================
# 3) USO DEL AGENTE – MODO BASE / PRUEBA SIMPLE (SIN MEMORIA)
# ============================================================
# OBJETIVO:
# Probar que un agente (creado por IU o por código) responde
# correctamente a UNA sola pregunta.
#
# IMPORTANTE:
# - No se reutiliza el contexto para múltiples turnos.
# - Aunque se cree una conversación, se usa solo una vez.
# - Conceptualmente, esto es equivalente a "stateless usage".
# ============================================================

import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

# ------------------------------------------------------------
# Conexión al proyecto de Azure AI Foundry
# Se reutiliza el endpoint configurado previamente
# ------------------------------------------------------------
project_client = AIProjectClient(
    endpoint=os.environ["AZURE_AI_FOUNDRY_PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

# ------------------------------------------------------------
# Cliente OpenAI-compatible para usar Responses y Conversations
# ------------------------------------------------------------
openai_client = project_client.get_openai_client()

# Paso opcional: Creamos una conversacion para usar el agente
conversation = openai_client.conversations.create()
print(f"ID de conversacion: {conversation.id}")

# ------------------------------------------------------------
# Envío de UNA pregunta al agente
# - Se usa un agente ya existente (creado por UI o código)
# - agent_reference evita recrear el agente
# - Este uso es SINGLE-TURN (sin continuidad)
# ------------------------------------------------------------
response = openai_client.responses.create(
    conversation=conversation.id, #Optional conversation context for multi-turn
    extra_body={"agent": {"name": agent_name, "type": "agent_reference"}},
    input="¿Cuál es el tamaño de Francia en millas cuadradas?",
)

# ------------------------------------------------------------
# Mostrar la respuesta del agente
# ------------------------------------------------------------
print(f"Respuesta: {response.output_text}")

ID de conversacion: conv_8e15a10ab49d09bd003MlzMudxzteuYHpgcOrABXeCUO23s7fd
Respuesta: No estoy habilitado para proporcionar información de temas distintos a matemáticas. ¿Quieres ayuda con algún problema o concepto matemático?


In [ ]:
# ============================================================
# 4) USO DEL AGENTE CON CONVERSACIÓN (MEMORIA DE SESIÓN)
# ============================================================
# NOTA: La "memoria" aquí es la conversación:
# Si usas el mismo conversation.id en múltiples turns,
# el agente podrá recordar lo que se habló antes (multi-turn).
# ============================================================
import os
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

# ------------------------------------------------------------
# Conexión al proyecto de Azure AI Foundry
# Se reutiliza el endpoint configurado previamente
# ------------------------------------------------------------
project_client = AIProjectClient(
    endpoint=os.environ["AZURE_AI_FOUNDRY_PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

# ------------------------------------------------------------
# Obtener el cliente OpenAI-compatible (Responses + Conversations)
# ------------------------------------------------------------
openai_client = project_client.get_openai_client()

# ------------------------------------------------------------
# Crear UNA conversación.
# IMPORTANTÍSIMO: Reutiliza conversation.id para mantener memoria.
# Si creas otra conversación, es como empezar de cero.
# ------------------------------------------------------------
conversation = openai_client.conversations.create()
print(f"ID de conversación: {conversation.id}")

ID de conversación: conv_6a0d1682277aac1d00BEtdsu4UU0e1UlgqExQEYOeIS8waZ8p3


In [ ]:
# -------------------------
# TURNO 1 (pregunta inicial)
# -------------------------
resp1 = openai_client.responses.create(
    conversation=conversation.id,  # <-- MISMA conversación = guarda contexto
    extra_body={"agent": {"name": agent_name, "type": "agent_reference"}},
    input="¿Cuánto es 3 + 2?"
)

print("Turno 1:", resp1.output_text)

Turno 1: 3 + 2 es igual a 5. ¿Necesitas ayuda con alguna otra operación?


In [ ]:
# -------------------------
# TURNO 2 (referencia al resultado previo)
# -------------------------
resp2 = openai_client.responses.create(
    conversation=conversation.id,  # <-- misma conversación = recuerda el turno 1
    extra_body={"agent": {"name": agent_name, "type": "agent_reference"}},
    input="Multiplica el resultado por 6."
)

print("Turno 2:", resp2.output_text)

Turno 2: El resultado de 5 multiplicado por 6 es 30. ¿Quieres que te ayude con otro cálculo?
